In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from backprojection.tools import sb1, sb1_r, sb1_tri, sb1_tri_r, sb2, wa

In [4]:
import sys

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [6]:
withPlots = 0

In [8]:
fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
c = 3e8
kc = 4 * np.pi * fc / c
fs = 10e6

rail = 0
truck = 1
plane = 2
vehicule = plane

if vehicule == rail: 
    # the objective is to have about 250 positions with an azimuth range of 3m, i.e. 0.012m steps
    # with T = 30e-3, this yields to V = 0.012 / 30e-3 = 0.4
    # 10 files correspond to 3m
    T = 30e-3
    rampsPerFile = 25
    V = 0.4 # 0.1
if vehicule == truck:
    T = 5e-3
    rampsPerFile = 200
    V = 3
if vehicule == plane:
    T = 600e-6
    rampsPerFile = 1500
    V = 40

alpha = 2 * (fMAX - fmin) / T
samplesPerRamp = int( np.ceil( T * fs ) )
samplesPerFile = rampsPerFile * samplesPerRamp
samplesPerUpRamp = int( samplesPerRamp / 2 )

print("T = {}, samplesPerRamp = {:.2f}, samplesPerFile = {:.2f}".format(T, samplesPerRamp, samplesPerFile))

t1 = np.arange( 0, T, 1/fs)
t2 = np.arange( -T/2, T/2, 1/fs)

T = 0.0006, samplesPerRamp = 6000.00, samplesPerFile = 9000000.00


# Build data

In [9]:
nbFiles = 20

In [10]:
a = 0.603354564402 # sinc(a) = 0.5
b = a * 180 / (30 * np.pi) # see the newton notebook for further details about the calculation

In [11]:
hPlane = 440
hCorner = 90
gr_corner = 350
d0_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5 # range of closest approach
az_corner = V * T * nbFiles * rampsPerFile / 2
noise = 1
print("d0_corner = {:.2f}, gr_corner = {:.2f}, az_corner = {:.2f}".format(d0_corner, gr_corner, az_corner))

d0_corner = 494.97, gr_corner = 350.00, az_corner = 360.00


In [12]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / d0_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 720, phi = 36.0°


In [14]:
sr_c = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp), dtype=complex )
sr_r = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp) )
for ramp in range( nbFiles * rampsPerFile):
    azn = ramp * T * V
    dn = ( d0_corner**2 + (azn - az_corner)**2 )**0.5
    noisen = np.random.randn( samplesPerRamp ) * noise / (dn**2)
    sr_c[ ramp,  0 : samplesPerRamp ] = sb1_tri( t2, dn, alpha, fc ) * wa( azn - az_corner, d0_corner, b ) \
    + noisen
    sr_r[ ramp,  0 : samplesPerRamp ] = sb1_tri_r( t2, dn, alpha, fc ) * wa( azn - az_corner, d0_corner, b ) \
    + noisen
    file = ramp/rampsPerFile
    if file == np.floor(file):
        phi = np.arccos( d0_corner / dn ) * 180 / np.pi
        print( "{}, azn = {:.2f}, wa = {:.1f}, phi = {:.1f}, dn = {:.1f}".format( 
            file, azn, wa( azn - az_corner, d0_corner ), phi, dn ) )

0.0, azn = 0.00, wa = 0.2, phi = 36.0, dn = 612.0
1.0, azn = 36.00, wa = 0.3, phi = 33.2, dn = 591.6
2.0, azn = 72.00, wa = 0.4, phi = 30.2, dn = 572.7
3.0, azn = 108.00, wa = 0.5, phi = 27.0, dn = 555.4
4.0, azn = 144.00, wa = 0.6, phi = 23.6, dn = 540.1
5.0, azn = 180.00, wa = 0.7, phi = 20.0, dn = 526.7
6.0, azn = 216.00, wa = 0.8, phi = 16.2, dn = 515.5
7.0, azn = 252.00, wa = 0.9, phi = 12.3, dn = 506.6
8.0, azn = 288.00, wa = 0.9, phi = 8.3, dn = 500.2
9.0, azn = 324.00, wa = 1.0, phi = 4.2, dn = 496.3
10.0, azn = 360.00, wa = 1.0, phi = 0.0, dn = 495.0
11.0, azn = 396.00, wa = 1.0, phi = 4.2, dn = 496.3
12.0, azn = 432.00, wa = 0.9, phi = 8.3, dn = 500.2
13.0, azn = 468.00, wa = 0.9, phi = 12.3, dn = 506.6
14.0, azn = 504.00, wa = 0.8, phi = 16.2, dn = 515.5
15.0, azn = 540.00, wa = 0.7, phi = 20.0, dn = 526.7
16.0, azn = 576.00, wa = 0.6, phi = 23.6, dn = 540.1
17.0, azn = 612.00, wa = 0.5, phi = 27.0, dn = 555.4
18.0, azn = 648.00, wa = 0.4, phi = 30.2, dn = 572.7
19.0, azn = 

In [15]:
sr_r0 = sr_r[0,:] * 2
sr_c0 = sr_c[0,:]
sr_r_b = np.real( sr_c ) * 2
sr_r0_b = sr_r_b[0,:]
if withPlots == 1:
    plt.figure()
    plt.plot( np.real( sr_r0 ), 'd-', label="sr_r0" )
    plt.plot( np.real( sr_r0_b ), '.-', label="sr_r0_b" )
    plt.grid()
    plt.legend()

Compare two ways of computing the real signal, by taking the real part of the complex signal $ sr_{rb} = real( sr_c ) * 2 $ or by computing directly in real $ sr_r $.

**Conclusion:** results are the same.

In [16]:
if withPlots == 1:
    plt.figure()
    plt.suptitle("ifft")
    plt.subplot(211)
    plt.plot( np.real( np.fft.ifft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.real( np.fft.ifft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.real( np.fft.ifft( sr_r0_b ) ), '.', label="sr_r0_b" )
    plt.grid()
    plt.legend()
    plt.subplot(212)
    plt.plot( np.imag( np.fft.ifft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.imag( np.fft.ifft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.imag( np.fft.ifft( sr_r0_b ) ), '.', label="sr_r0_b" )
    plt.grid()
    plt.legend()

In [17]:
sr_r0[10], sr_c0[10]*2, sr_r0_b[10]

(-0.17994937614073112,
 (-0.17994937614072545+0.13278479064592102j),
 -0.17994937614072545)

In [18]:
sa_r0 = signal.hilbert( sr_r0 ) / 2
if withPlots == 1:
    plt.figure()
    plt.suptitle("fft")
    plt.subplot(211)
    plt.plot( np.real( np.fft.fft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.real( np.fft.fft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.real( np.fft.fft( sa_r0 ) ), '.', label="sa_r0" )
    plt.grid()
    plt.legend()
    plt.subplot(212)
    plt.plot( np.imag( np.fft.fft( sr_r0 ) ), label="sr_r0" )
    plt.plot( np.imag( np.fft.fft( sr_c0 ) ), 'd', label="sr_c0" )
    plt.plot( np.imag( np.fft.fft( sa_r0 ) ), '.', label="sa_r0" )
    plt.grid()
    plt.legend()

Compare the complex signal computed using RD_realtocomp and the scipy hilbert complex conjugate.

**Conclusion:** results are the same.

In [19]:
sa_r0 = np.conjugate( signal.hilbert( sr_r0 ) )[::2]
RDc0 = RD_realtocomp( sr_r0 )
#RDc0 = RDc[0,:] * 2

In [20]:
if withPlots == 1:
    plt.figure()
    plt.suptitle("timeseries and ifft")
    ax1 = plt.subplot(221)
    plt.plot( np.real( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.real( RDc0 ), '.', label="RDc" )
    plt.grid()
    plt.legend()
    plt.subplot(222, sharex=ax1)
    plt.plot( np.imag( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.imag( RDc0 ), '.', label="RDc" )
    plt.grid()
    plt.legend()
    ax2 = plt.subplot(223)
    plt.plot( np.real( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.real( np.fft.ifft( RDc0 ) ), '.', label="RDc" )
    plt.grid()
    plt.legend()
    plt.subplot(224, sharex=ax2)
    plt.plot( np.imag( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.imag( np.fft.ifft( RDc0 ) ), '.', label="RDc" )
    plt.grid()

Compare the analytic signal computed with scipy hilbert complex conjugate and the complex signal $sr_c$.

**Conclusion:** little differences at the beginning of the series.

In [21]:
sa_r0 = np.conjugate( signal.hilbert( sr_r0 ) )
sr_c0 = sr_c[0,:]

In [22]:
if withPlots == 1:
    plt.figure()
    ax1 = plt.subplot(221)
    plt.plot( np.real( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.real( sr_c0*2 ), '.', label="sr_c0" )
    plt.grid()
    plt.legend()
    plt.subplot(222, sharex=ax1)
    plt.plot( np.imag( sa_r0 ), 'd-', label="sa_r0" )
    plt.plot( np.imag( sr_c0*2 ), '.', label="sr_c0" )
    plt.grid()
    plt.legend()
    ax2 = plt.subplot(223)
    plt.plot( np.real( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.real( np.fft.ifft( sr_c0*2 ) ), '.', label="RDc" )
    plt.grid()
    plt.legend()
    plt.subplot(224, sharex=ax2)
    plt.plot( np.imag( np.fft.ifft( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.imag( np.fft.ifft( sr_c0*2 ) ), '.', label="RDc" )
    plt.grid()

In [23]:
if withPlots == 1:
    plt.figure()
    ax1 = plt.subplot(211)
    plt.plot( 20 * np.log10( np.abs( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( 20 * np.log10( np.abs( sr_c0*2 ) ), 'xy', label="sr_c0" )
    plt.grid()
    plt.legend()
    plt.subplot(212, sharex=ax1)
    plt.plot( np.unwrap( np.angle( sa_r0 ) ), 'd-', label="sa_r0" )
    plt.plot( np.unwrap( np.angle( sr_c0*2 ) ), 'xy', label="sr_c0" )
    plt.grid()
    plt.legend()

# Save data

Keep only up ramps to build the complex incoming signal with numpy hilbert complex conjugate.

In [24]:
#complexSignal = np.conjugate(
#    signal.hilbert( sr_r[:, 0:samplesPerUpRamp], axis=1 )[:, ::2]
#)
complexSignal = sr_c[:, 0:samplesPerUpRamp:2]

In [25]:
srf = np.fft.ifft( complexSignal, axis=1 )
print( "srf.shape = {}".format(srf.shape))

srf.shape = (30000, 1500)


In [26]:
srf_dB = 20 * np.log10( np.abs( srf ) )

In [27]:
srf_min = np.amin(srf_dB)
srf_max = np.amax(srf_dB)
srf_med = np.median(srf_dB)
print("srf_min = {:.2f}, srf_max = {:.2f}, srf_med = {:.2f}".format(srf_min, srf_max, srf_med))

srf_min = -186.47, srf_max = -0.00, srf_med = -69.78


In [28]:
if withPlots == 1:
    plt.figure()
    plt.imshow( srf_dB[::10, 0:2000], cmap='jet' )

In [29]:
data_dir = "/home/pleroy/DATA/SIMU"
np.save( data_dir + "/complexSignalFrom_sr_c_nb{}_gr{}_T{}_rpf{}_V{}".format(
    nbFiles, gr_corner, T, rampsPerFile, V), complexSignal )

# Time series

In [ ]:
plt.figure()
r = 2
tau = 2 * r / c
print( "tau = {}, f = {}".format(tau, alpha*tau) )

ax = plt.subplot( 211 )
plt.plot( np.real( sb1( t1, r, alpha, fc ) ) )
plt.plot( np.real( sb1_tri( t1, r, alpha, fc ) ) )
plt.grid()

plt.subplot( 212, sharex=ax )
plt.plot( np.imag( sb1( t1, r, alpha, fc ) ) )
plt.plot( np.imag( sb1_tri( t1, r, alpha, fc ) ) )
plt.grid()

In [ ]:
t1.shape

In [ ]:
plt.figure()
r = 1000.5
tau = 2 * r / c
print( "tau = {}, f = {}".format(tau, alpha*tau) )
sb1_t1 = np.real( sb1( t1[0:3000], r ) )
sb2_t1 = np.real( sb2( t1[0:3000], r ) )
plt.subplot(311)
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb1_t1 ) ) ) )
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb2_t1 ) ) ) )
plt.subplot(312)
plt.plot( np.real(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.real(np.fft.ifft( sb2_t1 ) ) )
plt.subplot(313)
plt.plot( np.imag(np.fft.ifft( sb1_t1 ) ) )
plt.plot( np.imag(np.fft.ifft( sb2_t1 ) ) )